<a href="https://colab.research.google.com/github/EssamEmad/Continuous-Sign-Language-Recognition/blob/master/Chinese_Seq2Seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction

We are building and training a model that translates variable-length videos of Chinese Sign Language into variable-length Mandarin sentences.

The continuous sign language dataset we are training on has been collected by Haung et al and was moved to a google cloud bucket for quicker and easier downloads


In [0]:
#Uncomment the following lines to download the dataset
# !wget https://storage.googleapis.com/testingbucketforchinese/public_dataset.zip
# !unzip public_dataset.zip

--2019-06-03 17:26:48--  https://storage.googleapis.com/testingbucketforchinese/public_dataset.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.141.128, 2607:f8b0:400c:c06::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 56483925638 (53G) [application/zip]
Saving to: ‘public_dataset.zip’

public_dataset.zip   15%[==>                 ]   7.96G  16.2MB/s    eta 23m 9s ^C


In [0]:
# Configuration cell. Should be run only once on your vm instance
!sudo apt-get update
!sudo apt-get dist-upgrade
!sudo apt-get install ffmpeg
!sudo apt-get install -y \
    libavformat-dev libavcodec-dev libavdevice-dev \
    libavutil-dev libswscale-dev libswresample-dev libavfilter-dev
!pip3 install av
!git clone https://github.com/kenshohara/3D-ResNets-PyTorch.git
!pip3 install torch
!wget --header="Host: doc-10-a8-docs.googleusercontent.com" --header="User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3" --header="Accept-Language: en-US,en;q=0.9,ar;q=0.8" --header="Referer: https://drive.google.com/uc?id=1-4G8sxqBosFOp0x-jjlfBn5fIcEeQMyq&export=download" --header="Cookie: AUTH_lo9n4vqi8h6t6jtf9bq24tjt4mgiebjg_nonce=hnb6846hnul98; NID=179=AsVyYCkkBu7xhDR4X1VoRsL0G0XPY05E3nUFlZGIZPmBRL8iw27fHo5AMNpSi8Iplt3-SXHkuY6aRCI-DZRe_Jfi-Zl1EQT_VofK-JZFOyey3ckrYSYbVPZkCnJ3jh8W1b_tcmkA8c9HgOj4XyqWojsqt2E3kfa7qPm5kUxVgpA" --header="Connection: keep-alive" "https://doc-10-a8-docs.googleusercontent.com/docs/securesc/uedsrntu6as7p214rj0jj4fdfid709qm/ab5v08gsu6kvansu4jif3q8kmv2r4kob/1557489600000/06423779358691849595/09770380102039035018/1-4G8sxqBosFOp0x-jjlfBn5fIcEeQMyq?e=download&nonce=hnb6846hnul98&user=09770380102039035018&hash=t54obrrokd7ii0t0rp33sru694po9uhj" -O "best_params_100_64.pth" -c
!pip3 install torchvision
!pip3 install sk-video

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [801 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:13 http://archive.ubuntu.com/ubun

### Dataset Preparation

The continuous dataset consists of 100 gestures. Each gesture is repeated 50 times by 5 different signers. That is, there are 25,000 videos representing a 100 classes.

The dataset is prepared as follows:



*   Construct two dataframes: x_df containing the absolute paths of each videos and the corresponding y_df containing the sentence that represents this gesture
*   Get the target_tokens (List of unique words in all of the 100 sentences + #START and #END) and fit the labelencoder on them. (We could've just fit the labelencoder on all of the sentences, but we need the target_tokens for future use anyway)
* Construct *decoder_input_data* and *decoder_target_data*, which are the one-hot-encoded version of the target variable (sentences). Because we are using teacher-forcing method to train the decoder LSTM, the *decoder_target_data* is ahead of the *decoder_input_data* by 1 timestep. (Note that this step is skipped if we were to load the cached version of the dataset)



In [0]:
import pandas as pd
import os
sentences = {0:['他的', '同学', '是', '警察'],1:['他', '妈妈', '的', '同学', '是', '公务', '员'],2:['我的', '爸爸', '是', '商人'],3:['他', '哥哥', '的', '目标', '是', '解放', '军'],4:['他', '姐姐', '的', '目标', '是', '模特'],5:['我', '朋友', '的', '祖父', '是', '工人'],6:['我', '同学', '的', '妈妈', '是', '保姆'],7:['我', '同学', '的', '妹妹', '是', '护士'],8:['我的', '妻子', '是', '教师'],9:['我的', '丈夫', '是', '导演'],10:['他', '哥哥', '的', '朋友', '是', '演员'],11:['他的', '女朋友', '是', '护士'],12:['她', '丈夫', '的', '朋友', '是', '教练'],13:['他', '哥哥', '的', '同学', '是', '医生'],14:['他', '妹妹', '的', '同学', '是', '律师'],15:['他的', '邻居', '是', '残疾', '人'],16:['他的', '外祖母', '是', '园丁'],17:['他的', '祖父', '是', '炊事员'],18:['我', '表哥', '的', '邻居', '是', '记者'],19:['他的', '丈夫', '是', '警察'],20:['我', '公公', '是', '牧民'],21:['我', '婆婆', '是', '保育', '员'],22:['他', '儿子', '是', '弱', '智', '人'],23:['我', '嫂嫂', '是', '画家'],24:['你', '祖父', '是', '知识', '分子'],25:['你', '外祖父', '是', '猎', '手'],26:['他', '爸爸', '是', '保安'],27:['他', '妈妈', '是', '裁缝'],28:['他', '外祖父', '是', '邮递', '员'],29:['他', '祖母', '是', '聋人'],30:['我', '公公', '是', '门', '卫'],31:['你', '妹妹', '是', '会计'],32:['你', '哥哥', '是', '武', '警'],33:['你', '婆婆', '是', '盲人'],34:['你', '爸爸', '是', '编辑'],35:['她', '外祖父', '是', '农民'],36:['我', '儿子', '是', '职员'],37:['他', '弟弟', '是', '向导'],38:['他', '岳父', '是', '残疾', '人'],39:['他', '姐夫', '是', '刑警'],40:['我的', '新', '尺子', '是', '歪', '的'],41:['提高', '他的', '社会', '地位'],42:['紧张', '的', '工作', '气氛'],43:['工作', '效益', '稳定', '提高'],44:['工作', '环境', '的', '改善'],45:['民主', '团结', '的', '局势'],46:['我的', '毛毯', '是', '新的'],47:['现实', '情况', '我', '容易', '紧张'],48:['形成', '稳定', '的', '基础'],49:['社会', '团结', '是', '基础'],50:['我们', '的', '国家', '富强', '民主'],51:['国民', '的', '婚姻', '幸福'],52:['我们', '是', '自由', '恋爱'],53:['我们', '捐献', '的是', '毛毯'],54:['茶壶', '是', '褐色', '的'],55:['他的', '工作', '是', '美容'],56:['搞清', '形势', '是', '困难', '的'],57:['发挥', '我们', '的', '优势'],58:['扭转', '局面', '是', '困难', '的'],59:['现实', '是', '学生', '任务', '多'],60:['社会', '的', '安定'],61:['地球', '是', '行星'],62:['月亮', '是', '地球', '的', '卫星'],63:['太阳', '是', '恒星'],64:['国家', '经济', '情况', '好'],65:['他的', '前途', '事业', '成功'],66:['他们', '的', '国家', '摆脱', '贫苦'],67:['社会', '地位', '的', '提高'],68:['颜色', '是', '丰富', '的'],69:['社会', '就业', '形势', '好'],70:['他的', '工作', '经验', '丰富'],71:['剪刀', '是', '锋利', '的'],72:['新的', '被子', '是', '破', '的'],73:['天气', '预报', '有', '雨'],74:['他的', '盆', '是', '绿', '的'],75:['杯子', '是', '橙色', '的'],76:['加强', '国家', '的', '保卫'],77:['他', '放弃', '目标'],78:['引导', '他人', '成功'],79:['结果', '圆满', '成功'],80:['他的', '手表', '是', '坏', '的'],81:['他的', '牙刷', '疏'],82:['洗脸盆', '是', '空', '的'],83:['招聘', '的', '岗位', '多'],84:['我们', '拜访', '外祖父'],85:['事情', '有', '改善'],86:['我们', '的', '友谊', '深'],87:['裁缝', '有', '针线'],88:['妈妈', '有', '项链'],89:['我有', '打火', '机'],90:['天空', '没有', '星星'],91:['他的', '小孩子', '有', '礼貌'],92:['他', '招呼', '你来'],93:['观察', '他的', '情况'],94:['我', '推荐', '他', '去', '就业'],95:['手电筒', '有', '电池'],96:['我们', '的', '婚姻', '是', '幸福', '的'],97:['国家', '稳定', '是', '幸福', '的', '基础'],98:['社会', '地位', '是', '平等', '的'],99:['我的', '毛巾', '是', '干', '的']}

classes = [i for i in range(100)]
for k in list(sentences):
    if k not in set(classes):
        del sentences[k]
classes_col = []
paths_col = []
path = 'public_dataset/color'
for c in classes:
  paths = os.listdir('{}/{}/'.format(path,str(c).zfill(6)))
  paths = list(map(lambda x : '{}/{}/{}'.format(path,str(c).zfill(6),x), paths))
  classes_col += [c for i in range(len(paths))]
  paths_col += paths
y_df, x_df = pd.DataFrame(classes_col), pd.DataFrame(paths_col)

y_df = y_df.apply(lambda x : sentences[x[0]], axis=1)
x_df.shape, y_df.shape

FileNotFoundError: ignored

In [0]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
import functools
target_tokens = list(functools.reduce(lambda a,b : a.union(b), list(map(lambda x : set(x[1]), sentences.items()))))
target_tokens += ['#START', '#END']
num_classes = len(target_tokens)
sentences_array = list(map(lambda x : x[1], sentences.items()))
max_sentence = len(max(sentences_array, key=lambda x : len(x))) + 2
le = LabelEncoder()
le.fit(target_tokens)
len(target_tokens)

195

In [0]:
sentences_y = y_df.apply(lambda x : ['#START'] + x + ['#END'])
sentences_y_encoded = sentences_y.apply(lambda x :le.transform(x))
decoder_input_data = np.zeros((len(sentences_y_encoded),
                                    max_sentence, num_classes))
decoder_target_data = np.zeros((len(sentences_y_encoded),
                                    max_sentence, num_classes))
for i in range(decoder_input_data.shape[0]):
  for t in range(sentences_y_encoded[i].shape[0]):
    decoder_input_data[i,t, sentences_y_encoded[i][t]] = 1
    if t > 0:
      decoder_target_data[i,t-1, sentences_y_encoded[i][t]] = 1
decoder_target_data.shape, decoder_input_data.shape

LabelEncoder()

In [0]:
#build dict of indexes
classes = le.classes_
transformed = le.transform(classes)
decoder_indexes = dict((classes[i], transformed[i]) for i in range(len(classes)))
reverse_decoder_indexes = dict((transformed[i], classes[i]) for i in range(len(classes)))

({'#END': 0,
  '#START': 1,
  '丈夫': 2,
  '丰富': 3,
  '事业': 4,
  '事情': 5,
  '人': 6,
  '他': 7,
  '他人': 8,
  '他们': 9,
  '他的': 10,
  '任务': 11,
  '优势': 12,
  '会计': 13,
  '你': 14,
  '你来': 15,
  '保卫': 16,
  '保姆': 17,
  '保安': 18,
  '保育': 19,
  '儿子': 20,
  '公公': 21,
  '公务': 22,
  '军': 23,
  '农民': 24,
  '分子': 25,
  '刑警': 26,
  '前途': 27,
  '剪刀': 28,
  '加强': 29,
  '医生': 30,
  '卫': 31,
  '卫星': 32,
  '去': 33,
  '友谊': 34,
  '发挥': 35,
  '同学': 36,
  '向导': 37,
  '员': 38,
  '哥哥': 39,
  '商人': 40,
  '团结': 41,
  '园丁': 42,
  '困难': 43,
  '国家': 44,
  '国民': 45,
  '圆满': 46,
  '地位': 47,
  '地球': 48,
  '坏': 49,
  '基础': 50,
  '外祖母': 51,
  '外祖父': 52,
  '多': 53,
  '天气': 54,
  '天空': 55,
  '太阳': 56,
  '女朋友': 57,
  '她': 58,
  '好': 59,
  '妈妈': 60,
  '妹妹': 61,
  '妻子': 62,
  '姐夫': 63,
  '姐姐': 64,
  '婆婆': 65,
  '婚姻': 66,
  '嫂嫂': 67,
  '学生': 68,
  '安定': 69,
  '容易': 70,
  '富强': 71,
  '导演': 72,
  '小孩子': 73,
  '就业': 74,
  '尺子': 75,
  '局势': 76,
  '局面': 77,
  '岗位': 78,
  '岳父': 79,
  '工人': 80,
  '工作': 81,
  '干': 82,
  '平等': 83,
  '幸

In [0]:
from sklearn.model_selection import train_test_split
sentences_x_train, sentences_x_validation, decoder_input_train,\
decoder_input_val, decoder_target_train, decoder_target_val,\
sentences_y_train, sentences_y_validation=\
train_test_split(x_df, decoder_input_data, decoder_target_data, sentences_y,
                 test_size=0.2)


In [0]:
input_size = 112
sample_duration = 64
import torchvision
from torchvision import transforms
def validation_spatial_transform(frames, input_size):
    transform = transforms.Compose([
        transforms.CenterCrop(input_size*4),
        transforms.Resize(input_size),
        transforms.ToTensor(),
    ])
    return [transform(frame.to_image()) for frame in frames]
    

def training_spatial_transform(frames, input_size):
    """
    Applies one random transfomation to all of the frames
    of the same video.
    """
    translation = np.random.uniform(0, 0.07)
    crop_size = input_size*6#random.randrange(3, 7)
    # color jitter params
    b = np.random.uniform(0.7, 1.3)
    h = np.random.uniform(-0.2, 0.2)
    c = np.random.uniform(0.9, 1.1)
    s = np.random.uniform(0.8, 1.2)
    
    def transform(img):
        transforms.functional.affine(
            img,
            angle=0,
#             translate=(translation, translation),
            translate=(0,0),
            scale=1,
            shear=0,
        )
        img = transforms.CenterCrop(crop_size)(img)
        img = transforms.functional.resize(img, input_size)
        img = transforms.functional.adjust_brightness(img, b)
        img = transforms.functional.adjust_contrast(img, c)
        img = transforms.functional.adjust_hue(img, h)
        img = transforms.functional.adjust_saturation(img, s)
        tensor = transforms.functional.to_tensor(img)
        del img
        return tensor
        
    return [transform(frame.to_image()) for frame in frames]


In [0]:
import time
import keras
import cv2
import torch
import random
import av
import gc
import functools
import skvideo.io
import cv2
class SentenceGenerator(keras.utils.Sequence):
  
    """
    Generator for the continuous dataset generation. It transforms the video
    paths in x into its corresponding feature vectors by passing its
    corresponding frames onto the feature_extractor
    :param x: Dataframe containing the paths of the videos
    :param feature_extractor: Instance of Extractor class
    :param transform: function that is to be called to preprocess the video 
    before passing it onto the feature extractor (validation_spatial_transform
    or training_spatial_transform)
    :param cache: If set to True, the generator will cache the feature vectors
    in each iteration. The vector will be dumped onto {cache_path}/features.npy
    where cache_path is a global variable containing the cache folder name
    
    :Returns: A generator that yields ([features, decoder_input], decoder target)
    """
    def __init__(self, x, decoder_input, decoder_target, feature_extractor, 
                 transform,
               batch_size=4, cache=False):
        """
        :param x: video indexes
        :param y: label encoded sentences
        :param trimmed_cnn: feature extractor
        :param for_prediction: True if the generator is required to only return x 
        and not y as well
        """
        self.x = x
        self.decoder_input = decoder_input
        self.decoder_target = decoder_target
        self.feature_extractor = feature_extractor
        self.batch_size = batch_size
        self.cache = cache
        self.transform = transform
    
    def __get_video_frames__(self, vid):
        frames = [f for f in vid.decode(video=0)]
        return np.array(frames)
    def __len__(self):
        return len(self.x)


    def __transform__(self, frames):

        transformed_frames = self.transform(frames, 112)
        num_steps = len(transformed_frames)
        del frames
        torch.cuda.empty_cache()
        return torch.stack(transformed_frames).view(3, num_steps, 112, 112)
        
    def __pad_zeros__(self, features_list):
        max_length = max(features_list, key=lambda x : x.shape[0]).shape[0]
        for i in range(len(features_list)):
            zer = np.zeros((max_length,*features_list[0].shape[1:]))
            zer[:features_list[i].shape[0],:] = features_list[i].cpu() if type(features_list[i]) == torch.Tensor else features_list[i]
            features_list[i] = zer
        return features_list
    def __getitem__(self, index):
        """Reads the sequence of frames of each video and passes them through the 
        feature extractor"""
        segmentation_length = 64
        start = index
        videos = self.x.iloc[start: start + self.batch_size][0].values
        features_list = []
        for i, video_path in enumerate(videos):
            start_time = int(time.time())
            video = av.open(video_path)
            frames = self.__get_video_frames__(video)
            frames = self.__transform__(frames)
            features = []
            num_frames = frames.shape[1]
            for step in range(num_frames // segmentation_length):
                index = step * segmentation_length
                local_frames = frames[:,index: index+segmentation_length,:,:]
                local_frames = local_frames[np.newaxis,...]
                with torch.set_grad_enabled(False):
                    local_features = self.feature_extractor.extract(local_frames).flatten()
                    features.append(local_features)
                    del local_features
                    del local_frames
                    gc.collect()
                    torch.cuda.empty_cache()
            if num_frames % segmentation_length != 0:
                index = segmentation_length * (num_frames // segmentation_length)
                local_frames = frames[:,index:,:,:]
                old_shape = local_frames.shape
                zeros = np.zeros((3, segmentation_length, *old_shape[2:]))
                zeros[:, :local_frames.shape[1], :, :] = local_frames
                zeros = zeros[np.newaxis,:,:,:]
                with torch.set_grad_enabled(False):
                    features.append(self.feature_extractor.extract(torch.FloatTensor(zeros)).flatten())
                    del zeros
                    del frames
                    gc.collect()
                    torch.cuda.empty_cache()
            features = torch.stack(features)
            features_list.append(features)
            del features
            processing_time = int(time.time()) - start_time
            mins, seconds = processing_time // 60, processing_time % 60
            eta = processing_time * (len(videos) - i-1)
            if self.cache and i % 1000 == 0: #Every 100 videos we cache
                features_list = self.__pad_zeros__(features_list)
                np.save('{}/features'.format(cache_path), np.stack(features_list))
            print('Video {} processed in {}:{}\nETA: {}:{}'.format(i,mins, seconds,
                                                          eta // 60, eta %60 ))
            gc.collect()
            torch.cuda.empty_cache()
        features_list = self.__pad_zeros__(features_list)
        dec_input, dec_output = None, None
        if self.decoder_input is not None and self.decoder_target is not None:
          dec_input, dec_output = self.decoder_input\
        [start: start + self.batch_size, :, :], self.decoder_target[start: start+self.batch_size, :, :]

        return [np.stack(features_list), dec_input], dec_output#np.reshape(y, (y.shape[0], y.shape[1],1))

Using TensorFlow backend.


ModuleNotFoundError: ignored

In [0]:
import torch.nn as nn
class Extractor:
    def __init__(self, weights_path):
        self.weights_path = weights_path
        
    def load_model(self):
        model = resnet101(sample_size=112, sample_duration=64)
        model.fc = nn.Linear(model.fc.in_features, 100) # replace last fc nn layer to be able to load the weights
        model.load_state_dict(torch.load(self.weights_path))
        
        # remove last fc layer
        modules = list(model.children())
        self.fc = modules[-1]#.cuda()
        model = nn.Sequential(*modules[:-1])
        model.cuda()
        model.eval() # set the model to evaluation mode (instead of training mode)
        self.model = model

    def extract(self, video):
        assert self.model, 'no model loaded, please call load_model() first'
        if(video.shape[1:] != (3, 64, 112, 112)):
            raise ValueError('the shape of the sample video should be (batch_size, 3, 64, 112, 112).\
                             The shape received was {0}'.format(video.shape))
        
        video = video.cuda() # move input to gpu
        output = self.model(video) # feed forward through the network to calculate the outputs
        video = video.cpu()
        del video
        gc.collect()
        torch.cuda.empty_cache()
        return output
    
    def predict(self, video):
        output = self.extract(video)
        return self.fc(output.view(output.shape[0], self.fc.in_features))


In [0]:
gc.collect()
torch.cuda.empty_cache()
print('finished gc collect')
from resnets.models.resnext import resnet101
feature_extractor = Extractor('best_params_100_64.pth')
feature_extractor.load_model()

finished gc collect


/home/essam/Chinese/resnets/models/resnext.py:121: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')


Construct trained cnn to use as a feature extractor

In [0]:
#Uncomment out this cell to fetch the cache
# cache_path = 'cache_1'
# !wget https://storage.googleapis.com/chinesedatasetegslcanada/chinese-dataset.zip cache_1
# !mv chinese-dataset.zip cache_1/chinese_dataset.zip
# !unzip cache_1/chinese_dataset.zip -d cache_1/
# video = av.open(sentences_x_train.values[0][0])
# frames = sentence_generator.__get_video_frames__(video)
# transformed = sentence_generator.__transform__(frames)
# # import matplotlib.pyplot as plt
# plt.imshow(transformed[:,2,:,:].permute(1,2,0))

--2019-06-26 12:23:03--  https://storage.googleapis.com/chinesedatasetegslcanada/chinese-dataset.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.141.128, 2607:f8b0:400c:c06::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 729670558 (696M) [application/zip]
Saving to: ‘chinese-dataset.zip’

chinese-dataset.zip 100%[===================>] 695.87M   115MB/s    in 6.0s    

2019-06-26 12:23:09 (117 MB/s) - ‘chinese-dataset.zip’ saved [729670558/729670558]

--2019-06-26 12:23:09--  http://cache_1/
Resolving cache_1 (cache_1)... failed: Name or service not known.
wget: unable to resolve host address ‘cache_1’
FINISHED --2019-06-26 12:23:09--
Total wall clock time: 6.3s
Downloaded: 1 files, 696M in 6.0s (117 MB/s)
mv: cannot move 'chinese-dataset.zip' to 'cache_1/chinese_dataset.zip': No such file or directory
unzip:  cannot find or open cache_1/chinese_dataset.zip,

NameError: ignored

### Dataset Loading

The following cell loads and preprocesses the dataset. The videos are loaded using a generator that passes each video onto the feature extractor. The result is an array of sequences of feature vectors.

Since cache parameter is set to True in the SentenceGenerator, the accumulated feature vectors are cached in the cache_path in each iteration. This is done so as not to loose the processed data had the runtime been interrupted during execution of the function.

After the dataset is loaded, it's cached along with other information such as the target variable and the decoder inputs and outputs.

In [0]:
#Uncomment out this cell to load the dataset
# sentence_generator = SentenceGenerator(sentences_x_train, decoder_input_train,
#                                        decoder_target_train,
#                                        feature_extractor,
# transform=training_spatial_transform, batch_size=2, cache=True)
# np.save('{}/decoder_input'.format(cache_path), decoder_input_train)
# np.save('{}/decoder_output'.format(cache_path), decoder_target_train)
# np.save('{}/decoder_input_val'.format(cache_path), decoder_input_val)
# np.save('{}/decoder_target_val'.format(cache_path), decoder_target_val)
# np.save('{}/sentences_x_train'.format(cache_path), sentences_x_train)
# np.save('{}/sentences_x_val'.format(cache_path), sentences_x_validation)
# np.save('{}/sentences_y_train'.format(cache_path), sentences_y_train)
# np.save('{}/sentences_y_val'.format(cache_path),sentences_y_validation)
# sentence_generator.batch_size = decoder_input_train.shape[0]
# dataset = sentence_generator.__getitem__(0)
# np.save('{}/features'.format(cache_path), dataset[0][0])

In [0]:
#Load the cache
dataset = ([np.load('{}/features.npy'.format(cache_path)), np.load('{}/decoder_input.npy'.format(cache_path))], np.load('{}/decoder_output.npy'.format(cache_path)))
sentences_x_validation, decoder_input_val, decoder_target_val = \
pd.DataFrame(np.load('{}/sentences_x_val.npy'.format(cache_path))), np.load('{}/decoder_input_val.npy'.format(cache_path)), np.load('{}/decoder_target_val.npy'.format(cache_path))
sentences_y_validation = pd.DataFrame(np.load('{}/sentences_y_val.npy'.format(cache_path)))
sentences_y = pd.DataFrame(np.load('{}/sentences_y_train.npy'.format(cache_path)))


In [0]:
from keras.optimizers import RMSprop
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Masking
batch_size = 256  # Batch size for training.
epochs = 130  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_features = dataset[0][0].shape[2]
encoder_inputs = Input(shape=(None, num_features))
mask = Masking()(encoder_inputs)
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(mask)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_classes))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_classes, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer=RMSprop(), loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])
model.fit(dataset[0], dataset[1],
          epochs=epochs, verbose=1, batch_size=batch_size)

Epoch 1/130
20000/20000 [==============================] - 13s 636us/step - loss: 1.9901 - categorical_accuracy: 0.5732
Epoch 2/130
20000/20000 [==============================] - 11s 526us/step - loss: 1.5057 - categorical_accuracy: 0.6247
Epoch 3/130
20000/20000 [==============================] - 10s 522us/step - loss: 1.0155 - categorical_accuracy: 0.7525
Epoch 4/130
20000/20000 [==============================] - 10s 516us/step - loss: 0.6643 - categorical_accuracy: 0.8360
Epoch 5/130
20000/20000 [==============================] - 10s 515us/step - loss: 0.5200 - categorical_accuracy: 0.8531
Epoch 6/130
20000/20000 [==============================] - 10s 519us/step - loss: 0.4538 - categorical_accuracy: 0.8647
Epoch 7/130
20000/20000 [==============================] - 10s 514us/step - loss: 0.4130 - categorical_accuracy: 0.8740
Epoch 8/130
20000/20000 [==============================] - 10s 513us/step - loss: 0.3798 - categorical_accuracy: 0.8827
Epoch 9/130
20000/20000 [===============

20000/20000 [==============================] - 10s 502us/step - loss: 0.1170 - categorical_accuracy: 0.9612
Epoch 69/130
20000/20000 [==============================] - 10s 503us/step - loss: 0.1169 - categorical_accuracy: 0.9605
Epoch 70/130
20000/20000 [==============================] - 10s 508us/step - loss: 0.1149 - categorical_accuracy: 0.9620
Epoch 71/130
20000/20000 [==============================] - 10s 503us/step - loss: 0.1138 - categorical_accuracy: 0.9617
Epoch 72/130
20000/20000 [==============================] - 10s 508us/step - loss: 0.1131 - categorical_accuracy: 0.9623
Epoch 73/130
20000/20000 [==============================] - 10s 520us/step - loss: 0.1112 - categorical_accuracy: 0.9630
Epoch 74/130
20000/20000 [==============================] - 10s 512us/step - loss: 0.1089 - categorical_accuracy: 0.9633
Epoch 75/130
20000/20000 [==============================] - 10s 508us/step - loss: 0.1086 - categorical_accuracy: 0.9635
Epoch 76/130
20000/20000 [===================

In [0]:
#Cache the model
# model.save('{}/trained_model_256'.format(cache_path))

### Model Evaluation

We evaluate the model on the test dataset. For each test instance we do the following:

1. Feed the input sequence (Sequence of feature vectors representing the spatio-temporal features of the video) into the encoder LSTM

2. Feed the states + #START token into the decoder model and predict the next token

3. Keep sampling tokens until we predict #END or we reach the maximum sentence length (maximum number of tokens to be predicted)


###Accuracy Calculation

Our metric to evaluate the performance of the model is measured by calculating the minimum number of insertions, deletions, and replacements needed to transform the predicted sentence to the ground truth. This is calculated using the *min_transformations* function. The metric itself is calculated using:

accuracy = 1 - (S+D+I) / N

Where S, D, and I are the minimum number of replacements, deletions and insertions.

In [0]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_classes))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, decoder_indexes['#START']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
#         sampled_token_index = np.random.choice([i for i in range(output_tokens.shape[-1])], p=output_tokens[0, -1, :].ravel())
        sampled_token = reverse_decoder_indexes[sampled_token_index]
        decoded_sentence += ' {}'.format(sampled_token)

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_token == '#END' or
           len(decoded_sentence.split()) > max_sentence):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_classes))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence
def min_transformations(str1, str2, current_min=0):
  if not len(str2) or not len(str1):
    return current_min + len(str1) + len(str2)
  if str1[0] == str2[0]:
    return min_transformations(str1[1:], str2[1:], current_min)
  return min(min_transformations(str1[1:], str2[1:], current_min+1),
   min_transformations(str1, str2[1:], current_min+1), min_transformations(str1[1:], str2, current_min+1))

In [0]:
def pad_zeros(l):
    max_test_cache_len = max(l, key=lambda x: x.shape[1]).shape[1]
    for i in range(len(l)):
        zer = np.zeros((1,max_test_cache_len, 2048))
        zer[:,:l[i].shape[1],:] = l[i]
        l[i] = zer
    return l

(1, 9, 2048)

In [0]:
test_generator = SentenceGenerator(sentences_x_validation, None,
                                       None, transform=validation_spatial_transform,
                                       feature_extractor, batch_size=1)
test_cache = []
features_test = np.load('{}/features_test.npy'.format(cache_path))
av_acc = 0
num = 0
print('=' * 100)
print('Evaluating the Model')
print('=' * 100)
for i in range(0,5000):
    num += 1
    #Comment out the following lines to disable caching the test set
#     input_seq = test_generator.__getitem__(i)[0][0]
#     test_cache.append(input_seq)
#     if i % 100 == 0:
#         padded = pad_zeros(test_cache)
#         np.save('{}/features_test'.format(cache_path), np.stack(padded))
    input_seq = features_test[i]
    decoded_sentence = decode_sequence(input_seq).split()
    target = sentences_y_validation.values[i][0][1:]
    print('Actual:{}\nDecoded:{}'.format(target, decoded_sentence))
    min_tr = min_transformations(decoded_sentence, target)
    acc = 1 - (min_tr / len(target))
    av_acc = ((av_acc * (num-1)) + acc) / num
    print('===========\nIter:{}\nAcc:{}\tAV acc:{}\n==============================='.format(i, acc, av_acc))
# padded = pad_zeros(test_cache)
# np.save('{}/features_test'.format(cache_path), np.stack(padded))
print('=' * 100)
print('=' * 100)
print('Average Accuracy:{}'.format(av_acc))

DICK CHENEY MADE MONEY OFF THE IRAQ WAR
Actual:['引导', '他人', '成功', '#END']
Decoded:['引导', '他人', '成功', '#END']
Iter:0
Acc:1.0	AV acc:1.0
Actual:['加强', '国家', '的', '保卫', '#END']
Decoded:['加强', '国家', '的', '保卫', '#END']
Iter:1
Acc:1.0	AV acc:1.0
Actual:['你', '外祖父', '是', '猎', '手', '#END']
Decoded:['他的', '工作', '经验', '丰富', '#END']
Iter:2
Acc:0.16666666666666663	AV acc:0.7222222222222222
Actual:['我', '表哥', '的', '邻居', '是', '记者', '#END']
Decoded:['我', '表哥', '的', '邻居', '是', '记者', '#END']
Iter:3
Acc:1.0	AV acc:0.7916666666666666
Actual:['国家', '稳定', '是', '幸福', '的', '基础', '#END']
Decoded:['国家', '稳定', '是', '幸福', '的', '基础', '#END']
Iter:4
Acc:1.0	AV acc:0.8333333333333333
Actual:['我们', '的', '友谊', '深', '#END']
Decoded:['我们', '的', '友谊', '深', '#END']
Iter:5
Acc:1.0	AV acc:0.861111111111111
Actual:['工作', '效益', '稳定', '提高', '#END']
Decoded:['工作', '效益', '稳定', '提高', '#END']
Iter:6
Acc:1.0	AV acc:0.8809523809523808
Actual:['国民', '的', '婚姻', '幸福', '#END']
Decoded:['国民', '的', '婚姻', '幸福', '#END']
Iter:7
Acc:1.0	AV a

Actual:['紧张', '的', '工作', '气氛', '#END']
Decoded:['紧张', '的', '工作', '气氛', '#END']
Iter:52
Acc:1.0	AV acc:0.8029200359389037
Actual:['他的', '工作', '经验', '丰富', '#END']
Decoded:['他的', '工作', '经验', '丰富', '#END']
Iter:53
Acc:1.0	AV acc:0.8065696649029981
Actual:['发挥', '我们', '的', '优势', '#END']
Decoded:['他的', '邻居', '是', '残疾', '人', '#END']
Iter:54
Acc:0.0	AV acc:0.7919047619047618
Actual:['观察', '他的', '情况', '#END']
Decoded:['观察', '他的', '情况', '#END']
Iter:55
Acc:1.0	AV acc:0.7956207482993196
Actual:['他的', '牙刷', '疏', '#END']
Decoded:['他的', '牙刷', '疏', '#END']
Iter:56
Acc:1.0	AV acc:0.799206349206349
Actual:['他', '儿子', '是', '弱', '智', '人', '#END']
Decoded:['他', '儿子', '是', '弱', '智', '人', '#END']
Iter:57
Acc:1.0	AV acc:0.8026683087027913
Actual:['我', '公公', '是', '门', '卫', '#END']
Decoded:['我', '公公', '是', '门', '卫', '#END']
Iter:58
Acc:1.0	AV acc:0.8060129136400321
Actual:['我', '推荐', '他', '去', '就业', '#END']
Decoded:['我', '推荐', '他', '去', '就业', '#END']
Iter:59
Acc:1.0	AV acc:0.8092460317460316
Actual:['他的', '丈夫'

Actual:['洗脸盆', '是', '空', '的', '#END']
Decoded:['洗脸盆', '是', '空', '的', '#END']
Iter:104
Acc:1.0	AV acc:0.7765079365079366
Actual:['引导', '他人', '成功', '#END']
Decoded:['引导', '他人', '成功', '#END']
Iter:105
Acc:1.0	AV acc:0.778616352201258
Actual:['结果', '圆满', '成功', '#END']
Decoded:['结果', '圆满', '成功', '#END']
Iter:106
Acc:1.0	AV acc:0.7806853582554518
Actual:['我们', '捐献', '的是', '毛毯', '#END']
Decoded:['我们', '捐献', '的是', '毛毯', '#END']
Iter:107
Acc:1.0	AV acc:0.7827160493827162
Actual:['太阳', '是', '恒星', '#END']
Decoded:['太阳', '是', '恒星', '#END']
Iter:108
Acc:1.0	AV acc:0.7847094801223242
Actual:['他', '祖母', '是', '聋人', '#END']
Decoded:['他', '祖母', '是', '聋人', '#END']
Iter:109
Acc:1.0	AV acc:0.7866666666666667
Actual:['他的', '牙刷', '疏', '#END']
Decoded:['他', '妈妈', '是', '裁缝', '#END']
Iter:110
Acc:0.0	AV acc:0.7795795795795797
Actual:['我', '儿子', '是', '职员', '#END']
Decoded:['我', '儿子', '是', '职员', '#END']
Iter:111
Acc:1.0	AV acc:0.7815476190476192
Actual:['我', '公公', '是', '门', '卫', '#END']
Decoded:['我', '公公', '是', '

Actual:['社会', '团结', '是', '基础', '#END']
Decoded:['社会', '团结', '是', '基础', '#END']
Iter:154
Acc:1.0	AV acc:0.792657450076805
Actual:['我们', '的', '婚姻', '是', '幸福', '的', '#END']
Decoded:['手电筒', '有', '电池', '#END']
Iter:155
Acc:0.1428571428571429	AV acc:0.7884920634920635
Actual:['发挥', '我们', '的', '优势', '#END']
Decoded:['社会', '团结', '是', '基础', '#END']
Iter:156
Acc:0.19999999999999996	AV acc:0.7847437063997574
Actual:['他', '儿子', '是', '弱', '智', '人', '#END']
Decoded:['他', '儿子', '是', '弱', '智', '人', '#END']
Iter:157
Acc:1.0	AV acc:0.7861060880048223
Actual:['颜色', '是', '丰富', '的', '#END']
Decoded:['颜色', '是', '丰富', '的', '#END']
Iter:158
Acc:1.0	AV acc:0.7874513327343517
Actual:['我的', '妻子', '是', '教师', '#END']
Decoded:['我的', '妻子', '是', '教师', '#END']
Iter:159
Acc:1.0	AV acc:0.7887797619047621
Actual:['我', '婆婆', '是', '保育', '员', '#END']
Decoded:['我', '婆婆', '是', '保育', '员', '#END']
Iter:160
Acc:1.0	AV acc:0.790091688849453
Actual:['他的', '手表', '是', '坏', '的', '#END']
Decoded:['他的', '手表', '是', '坏', '的', '#END']
Ite

Actual:['他', '姐夫', '是', '刑警', '#END']
Decoded:['他', '姐夫', '是', '刑警', '#END']
Iter:205
Acc:1.0	AV acc:0.7791377716134997
Actual:['现实', '情况', '我', '容易', '紧张', '#END']
Decoded:['我', '公公', '是', '门', '卫', '#END']
Iter:206
Acc:0.16666666666666663	AV acc:0.7761789740050609
Actual:['国民', '的', '婚姻', '幸福', '#END']
Decoded:['国民', '的', '婚姻', '幸福', '#END']
Iter:207
Acc:1.0	AV acc:0.7772550366300366
Actual:['裁缝', '有', '针线', '#END']
Decoded:['裁缝', '有', '针线', '#END']
Iter:208
Acc:1.0	AV acc:0.7783208020050124
Actual:['你', '爸爸', '是', '编辑', '#END']
Decoded:['你', '爸爸', '是', '编辑', '#END']
Iter:209
Acc:1.0	AV acc:0.77937641723356
Actual:['我们', '是', '自由', '恋爱', '#END']
Decoded:['我们', '是', '自由', '恋爱', '#END']
Iter:210
Acc:1.0	AV acc:0.7804220266305574
Actual:['手电筒', '有', '电池', '#END']
Decoded:['工作', '环境', '的', '改善', '#END']
Iter:211
Acc:0.0	AV acc:0.776740790655885
Actual:['你', '爸爸', '是', '编辑', '#END']
Decoded:['杯子', '是', '橙色', '的', '#END']
Iter:212
Acc:0.19999999999999996	AV acc:0.774033087413369
Actual:['他

...

Actual:['我', '嫂嫂', '是', '画家', '#END']
Decoded:['你', '祖父', '是', '知识', '分子', '#END']
Iter:4790
Acc:0.19999999999999996	AV acc:0.7321652205027273
Actual:['我', '儿子', '是', '职员', '#END']
Decoded:['提高', '他的', '社会', '地位', '#END']
Iter:4791
Acc:0.19999999999999996	AV acc:0.7320541676603852
Actual:['结果', '圆满', '成功', '#END']
Decoded:['我们', '的', '婚姻', '是', '幸福', '的', '#END']
Iter:4792
Acc:-0.5	AV acc:0.731797114840093
Actual:['他', '妈妈', '是', '裁缝', '#END']
Decoded:['他', '妈妈', '是', '裁缝', '#END']
Iter:4793
Acc:1.0	AV acc:0.7318530603730843
Actual:['社会', '就业', '形势', '好', '#END']
Decoded:['社会', '就业', '形势', '好', '#END']
Iter:4794
Acc:1.0	AV acc:0.7319089825711295
Actual:['民主', '团结', '的', '局势', '#END']
Decoded:['形成', '稳定', '的', '基础', '#END']
Iter:4795
Acc:0.4	AV acc:0.7318397771952807
Actual:['我', '公公', '是', '门', '卫', '#END']
Decoded:['我的', '丈夫', '是', '导演', '#END']
Iter:4796
Acc:0.33333333333333337	AV acc:0.7317567030981655
Actual:['太阳', '是', '恒星', '#END']
Decoded:['太阳', '是', '恒星', '#END']
Iter:4797
Acc:

Iter:4841
Acc:1.0	AV acc:0.7313725634822277
Actual:['加强', '国家', '的', '保卫', '#END']
Decoded:['我们', '捐献', '的是', '毛毯', '#END']
Iter:4842
Acc:0.19999999999999996	AV acc:0.7312628437705857
Actual:['裁缝', '有', '针线', '#END']
Decoded:['裁缝', '有', '针线', '#END']
Iter:4843
Acc:1.0	AV acc:0.7313183221265372
Actual:['我的', '爸爸', '是', '商人', '#END']
Decoded:['我的', '爸爸', '是', '商人', '#END']
Iter:4844
Acc:1.0	AV acc:0.7313737775812067
Actual:['他', '祖母', '是', '聋人', '#END']
Decoded:['他', '祖母', '是', '聋人', '#END']
Iter:4845
Acc:1.0	AV acc:0.7314292101487715
Actual:['他的', '同学', '是', '警察', '#END']
Decoded:['他', '岳父', '是', '残疾', '人', '#END']
Iter:4846
Acc:0.19999999999999996	AV acc:0.7313195692966673
Actual:['他的', '前途', '事业', '成功', '#END']
Decoded:['他的', '工作', '是', '美容', '#END']
Iter:4847
Acc:0.4	AV acc:0.7312512278013503
Actual:['我', '朋友', '的', '祖父', '是', '工人', '#END']
Decoded:['我的', '爸爸', '是', '商人', '#END']
Iter:4848
Acc:0.2857142857142857	AV acc:0.731159345569532
Actual:['我们', '的', '婚姻', '是', '幸福', '的', '#END'

Actual:['他们', '的', '国家', '摆脱', '贫苦', '#END']
Decoded:['他们', '的', '国家', '摆脱', '贫苦', '#END']
Iter:4893
Acc:1.0	AV acc:0.7315943234670236
Actual:['他的', '丈夫', '是', '警察', '#END']
Decoded:['他的', '盆', '是', '绿', '的', '#END']
Iter:4894
Acc:0.4	AV acc:0.7315265820321989
Actual:['裁缝', '有', '针线', '#END']
Decoded:['裁缝', '有', '针线', '#END']
Iter:4895
Acc:1.0	AV acc:0.7315814172891367
Actual:['我的', '新', '尺子', '是', '歪', '的', '#END']
Decoded:['我的', '新', '尺子', '是', '歪', '的', '#END']
Iter:4896
Acc:1.0	AV acc:0.7316362301506256
Actual:['社会', '的', '安定', '#END']
Decoded:['社会', '的', '安定', '#END']
Iter:4897
Acc:1.0	AV acc:0.7316910206303825
Actual:['社会', '地位', '的', '提高', '#END']
Decoded:['社会', '地位', '的', '提高', '#END']
Iter:4898
Acc:1.0	AV acc:0.7317457887421134
Actual:['我们', '拜访', '外祖父', '#END']
Decoded:['我们', '拜访', '外祖父', '#END']
Iter:4899
Acc:1.0	AV acc:0.731800534499513
Actual:['他', '招呼', '你来', '#END']
Decoded:['他', '招呼', '你来', '#END']
Iter:4900
Acc:1.0	AV acc:0.7318552579162647
Actual:['工作', '效益', '稳定', '提

Actual:['他的', '丈夫', '是', '警察', '#END']
Decoded:['我的', '妻子', '是', '教师', '#END']
Iter:4944
Acc:0.4	AV acc:0.7310185853916888
Actual:['你', '祖父', '是', '知识', '分子', '#END']
Decoded:['我们', '捐献', '的是', '毛毯', '#END']
Iter:4945
Acc:0.16666666666666663	AV acc:0.7309044826988613
Actual:['你', '妹妹', '是', '会计', '#END']
Decoded:['你', '妹妹', '是', '会计', '#END']
Iter:4946
Acc:1.0	AV acc:0.7309588783967188
Actual:['现实', '是', '学生', '任务', '多', '#END']
Decoded:['他', '姐姐', '的', '目标', '是', '模特', '#END']
Iter:4947
Acc:0.0	AV acc:0.7308111502482959
Actual:['形成', '稳定', '的', '基础', '#END']
Decoded:['形成', '稳定', '的', '基础', '#END']
Iter:4948
Acc:1.0	AV acc:0.7308655428225032
Actual:['他', '妈妈', '是', '裁缝', '#END']
Decoded:['他', '妈妈', '是', '裁缝', '#END']
Iter:4949
Acc:1.0	AV acc:0.7309199134199127
Actual:['我们', '拜访', '外祖父', '#END']
Decoded:['社会', '团结', '是', '基础', '#END']
Iter:4950
Acc:0.0	AV acc:0.7307722826557398
Actual:['我', '公公', '是', '门', '卫', '#END']
Decoded:['天气', '预报', '有', '雨', '#END']
Iter:4951
Acc:0.1666666666666

Actual:['工作', '环境', '的', '改善', '#END']
Decoded:['工作', '环境', '的', '改善', '#END']
Iter:4994
Acc:1.0	AV acc:0.7309011392344714
Actual:['发挥', '我们', '的', '优势', '#END']
Decoded:['他', '姐姐', '的', '目标', '是', '模特', '#END']
Iter:4995
Acc:0.0	AV acc:0.730754841968812
Actual:['我', '表哥', '的', '邻居', '是', '记者', '#END']
Decoded:['我', '婆婆', '是', '保育', '员', '#END']
Iter:4996
Acc:0.2857142857142857	AV acc:0.7306657804206322
Actual:['社会', '地位', '的', '提高', '#END']
Decoded:['扭转', '局面', '是', '困难', '的', '#END']
Iter:4997
Acc:0.0	AV acc:0.730519588787895
Actual:['他的', '小孩子', '有', '礼貌', '#END']
Decoded:['他', '祖母', '是', '聋人', '#END']
Iter:4998
Acc:0.19999999999999996	AV acc:0.7304134636451088
Actual:['我的', '爸爸', '是', '商人', '#END']
Decoded:['我的', '爸爸', '是', '商人', '#END']
Iter:4999
Acc:1.0	AV acc:0.7304673809523798
